<a href="https://colab.research.google.com/github/baldan555/capstone/blob/main/TireBinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dataset_path = ("/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH//256/train")
validation_dataset_path = ("/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/256/test")

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.28,
                                  shear_range=0.3,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size= (IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                   color_mode ='rgb',
                                                   class_mode='binary',
                                                   shuffle=True)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255,
                                        )
validation_generator = validation_datagen.flow_from_directory(validation_dataset_path,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                             color_mode ='rgb',
                                                             class_mode='binary',
                                                             shuffle=True)

In [ ]:
sample_train_images, _ = next(train_generator)
print(sample_train_images.shape)

In [ ]:
sample_valid_images, _ = next(validation_generator)
print(sample_valid_images.shape)

In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label class dari data train\n")
for key, value in labels.items():
    print(f"{key} : {value}")

In [ ]:
model = Sequential([

tf.keras.layers.Conv2D(16, (5,5), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001,
                 decay = 1e-4,
                 beta_1=0.8,
                 beta_2=0.999,
                 amsgrad=True)

In [ ]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)


"""class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")
        train_accuracy = logs.get("accuracy")
        if val_accuracy > self.threshold and train_accuracy > self.threshold:
            self.model.stop_training = True

callbacks = MyThresholdCallback(threshold=0.92)"""


In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_generator, epochs=60, validation_data=validation_generator,
                       verbose=2,
                       
                       )

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']



In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 10))

ax[0].set_title('Training Accuracy vs. Epochs')
ax[0].plot(train_accuracy, 'o-', label='Train Accuracy')
ax[0].plot(val_accuracy, 'o-', label='Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(loc='best')

ax[1].set_title('Training/Validation Loss vs. Epochs')
ax[1].plot(train_loss, 'o-', label='Train Loss')
ax[1].plot(val_loss, 'o-', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(loc='best')



plt.tight_layout()
plt.show()

In [ ]:
test_dataset_path = ("/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/coba")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(test_dataset_path,
                                                 shuffle=False,
                                                 batch_size=BATCH_SIZE,
                                                 target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                 class_mode='binary')

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(10, 8))
idx = 0

for i in range(2):
    for j in range(5):
        predicted_label = labels[np.argmax(predictions[idx])]
        ax[i, j].set_title(f"{predicted_label}")
        ax[i, j].imshow(test_generator[0][0][idx])
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Test Dataset Predictions", fontsize=20, color = 'red')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, batch_size=BATCH_SIZE)

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

uploaded = files.upload()

for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(256, 256))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=32)

    print(fn)
    if classes[0, 0] == 1.0:
        print('Kualitas buruk')
    else:
        print('Kualitas baik')


In [ ]:
from tensorflow.keras.models import load_model
import os


In [ ]:
model.save(os.path.join('drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/file model', 'Tirebinary.h5'))

In [ ]:
model = keras.models.load_model('drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/file model/Tirebinary.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/file model/Tirebinary.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
model_json = model.to_json()

In [ ]:
with open('drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/file model/Tirebinary.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
model.save('/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/file model/saved_model')